# Random Forest

Boostrap Sample: 데이터 세트에서 중복을 허용하여 데이터를 샘플링하는 방식

RandomForestClassifier는 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 무작위 선택함. (*회귀 모델인 RandomForestRegressor는 전체 특성을 사용)

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state=42)

In [5]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score = True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [9]:
print(scores)

{'fit_time': array([0.17181778, 0.20305896, 0.20305896, 0.20305896, 0.29677773]), 'score_time': array([0.03123975, 0.03123999, 0.03123999, 0.03123999, 0.01562023]), 'test_score': array([0.88461538, 0.88942308, 0.90279115, 0.88931665, 0.88642926]), 'train_score': array([0.9971133 , 0.99663219, 0.9978355 , 0.9973545 , 0.9978355 ])}


In [6]:
# random forest의 중요도는 각 decision tree의 중요도를 취합한 것임.
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [11]:
# 부트스트랩 샘플에 포함되지 않고 남는 샘플들을 OOB(out of bag) 샘플이라고 함.
# 이 샘플들을 검증 세트처럼 사용할 수 있음.
rf = RandomForestClassifier(oob_score = True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)    # 교차 검증의 테스트셋 점수와 비슷한 결과를 얻을 수 있음.

0.8934000384837406


# Extra Tree

random forest와 다르게 bootstrap 샘플을 사용하지 않음. 즉 각 결정 트리를 만들 때 전체 훈련 세트를 사용함. 대신 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할함.

In [17]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state = 42)
scores = cross_validate(et,train_input, train_target, return_train_score = True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


이 실습에서는 랜덤 포레스트와 비슷한 결과가 나타나지만, 이 예제는 특성이 많지 않아 두 모델의 차이가 크지 않음. 보통 엑스트라 트리가 무작위성이 좀 더 크기 때문에 랜덤 포레스트보다 더 많은 결정 트리를 훈련해야함. 하지만 랜덤하게 노드를 분할하기 때문에 빠른 계산 속도가 엑스트라 트리의 장점임.

In [18]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


# Gradient Boosting

default는 depth가 3인 decision tree를 100개 사용함. 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대할 수 있음.

경사하강법을 사용하여 트리를 앙상블에 추가함. 분류에서는 로지스틱 손실 함수를 사용하고 회귀에서는 평균 제곱 오차 함수를 사용함.

In [19]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [21]:
gb = GradientBoostingClassifier(n_estimators = 500, learning_rate = 0.2, random_state = 42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [22]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15853457 0.68010884 0.1613566 ]


# Histogram-based Gradient Boosting

입력 특성을 256개 구간으로 나눔. 따라서 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있음. 히스토그램 기반 그레이디언트 부스팅은 256개의 구간 중에서 하나를 떼어 놓고 누락된 값을 위해서 사용함. 따라서 입력에 누락된 특성이 있더라도 이를 따로 전처리할 필요가 없음.

In [26]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score = True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [27]:
# Histogram-based Gradient Boositng -> 특성 중요도 계산: permutation_importance()
# n_repeats: 랜덤하게 섞을 횟수 지정 (Default: 5)
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats = 10, random_state = 42, n_jobs = -1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [29]:
result = permutation_importance(hgb, test_input, test_target, n_repeats = 10, random_state = 42, n_jobs = -1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [30]:
hgb.score(test_input, test_target)

0.8723076923076923

In [32]:
pip install xgboost


     --------------------------------------- 89.1/89.1 MB 17.2 MB/s eta 0:00:00


In [33]:
# sklearn 외에 그레이디언트 부스팅 알고리즘을 구현한 라이브러리 예제
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method = 'hist', random_state = 42)
scores = cross_validate(xgb, train_input, train_target, return_train_score = True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


In [35]:
pip install lightgbm

     ---------------------------------------- 1.0/1.0 MB 9.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [37]:
# sklearn 외에 그레이디언트 부스팅 알고리즘을 구현한 라이브러리 예제
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state = 42)
scores = cross_validate(lgb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
